In [28]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [29]:
# Definir las rutas de las carpetas de entrenamiento y prueba
train_folder = 'CarneDataset/train'
test_folder = 'CarneDataset/test'

In [30]:
# Leer las imágenes de entrenamiento y sus etiquetas
train_images = []
train_labels = []
for label in os.listdir(train_folder):
    label_folder = os.path.join(train_folder, label)
    for filename in os.listdir(label_folder):
        img_path = os.path.join(label_folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            train_images.append(img)
            train_labels.append(label)

In [31]:
# Convertir las listas de imágenes y etiquetas a matrices NumPy
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [32]:
# Codificar las etiquetas como números
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)

In [33]:
# Dividir el conjunto de entrenamiento en entrenamiento y validación
train_images, val_images, train_labels_encoded, val_labels_encoded = train_test_split(train_images, train_labels_encoded, test_size=0.2, random_state=42)

In [34]:
# Normalizar los valores de píxeles entre 0 y 1
train_images = train_images.astype('float32') / 255.0
val_images = val_images.astype('float32') / 255.0

In [35]:
# Definir la arquitectura de la CNN
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=train_images.shape[1:]))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [36]:
# Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [37]:
# Entrenar el modelo
model.fit(train_images, train_labels_encoded, epochs=10, batch_size=32, validation_data=(val_images, val_labels_encoded))

Epoch 1/10
41/41 [==============================] - 63s 2s/step - loss: 1.9417 - accuracy: 0.5884 - val_loss: 0.7680 - val_accuracy: 0.7003
Epoch 2/10
41/41 [==============================] - 61s 1s/step - loss: 0.6283 - accuracy: 0.7972 - val_loss: 0.4556 - val_accuracy: 0.8073
Epoch 3/10
41/41 [==============================] - 61s 1s/step - loss: 0.3890 - accuracy: 0.8692 - val_loss: 0.5631 - val_accuracy: 0.7645
Epoch 4/10
41/41 [==============================] - 61s 1s/step - loss: 0.1869 - accuracy: 0.9449 - val_loss: 0.3313 - val_accuracy: 0.8838
Epoch 5/10
41/41 [==============================] - 60s 1s/step - loss: 0.1098 - accuracy: 0.9740 - val_loss: 0.3518 - val_accuracy: 0.8838
Epoch 6/10
41/41 [==============================] - 61s 1s/step - loss: 0.1032 - accuracy: 0.9640 - val_loss: 0.3846 - val_accuracy: 0.8807
Epoch 7/10
41/41 [==============================] - 61s 1s/step - loss: 0.0718 - accuracy: 0.9770 - val_loss: 0.3513 - val_accuracy: 0.8991
Epoch 8/10
41/41 [==

In [38]:
test_images = []
test_labels = []
for label in os.listdir(test_folder):
    label_folder = os.path.join(test_folder, label)
    for filename in os.listdir(label_folder):
        img_path = os.path.join(label_folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            test_images.append(img)
            test_labels.append(label)

In [39]:
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [40]:
test_images = test_images.astype('float32') / 255.0

In [41]:
test_labels_encoded = label_encoder.transform(test_labels)

In [42]:
predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)

26/26 [==============================] - 9s 331ms/step


In [43]:
# Calcular la matriz de confusión del modelo
confusion = confusion_matrix(test_labels_encoded, predicted_labels)
print('Matriz de confusión del modelo:')
print(confusion)

Matriz de confusión del modelo:
[[  1   0   0   0   0   0   0   0]
 [  0  27   0  17   0   0   4   0]
 [  0   0  82   2   5   1   7   0]
 [  0   3   1  40   0   0   1   0]
 [  0   0   6   1 442   0   3   7]
 [  0   0   0   2   2  15   0   0]
 [  0   2  16   4   9   0  83   0]
 [  0   0   0   0   7   0   0  20]]


In [44]:
# Calcular la matriz de confusión del error en entrenamiento
train_predictions = model.predict(train_images)
train_predicted_labels = np.argmax(train_predictions, axis=1)
train_confusion = confusion_matrix(train_labels_encoded, train_predicted_labels)
print('Matriz de confusión del error en entrenamiento:')
print(train_confusion)

41/41 [==============================] - 13s 320ms/step
Matriz de confusión del error en entrenamiento:
[[  1   0   0   0   0   0   0   0]
 [  0  51   0   0   0   0   0   0]
 [  0   0 165   0   0   0   0   0]
 [  0   0   0  82   0   0   0   0]
 [  0   0   0   0 761   0   0   0]
 [  0   0   0   0   0  30   0   0]
 [  0   0   0   0   0   0 170   0]
 [  0   0   0   0   0   0   0  47]]


In [45]:
# Calcular la matriz de confusión del error en prueba
test_predictions = model.predict(test_images)
test_predicted_labels = np.argmax(test_predictions, axis=1)
test_confusion = confusion_matrix(test_labels_encoded, test_predicted_labels)
print('Matriz de confusión del error en prueba:')
print(test_confusion)

26/26 [==============================] - 8s 313ms/step
Matriz de confusión del error en prueba:
[[  1   0   0   0   0   0   0   0]
 [  0  27   0  17   0   0   4   0]
 [  0   0  82   2   5   1   7   0]
 [  0   3   1  40   0   0   1   0]
 [  0   0   6   1 442   0   3   7]
 [  0   0   0   2   2  15   0   0]
 [  0   2  16   4   9   0  83   0]
 [  0   0   0   0   7   0   0  20]]
